In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from unsloth import FastModel
from peft import PeftModel
import torch

# Paso 1: carga el modelo base
base_model_name = "unsloth/gemma-3-4b-it"
#lora_path = "/content/drive/MyDrive/MOE_DGA/outputs_gemma_2epoch/model"
lora_path = "/content/drive/MyDrive/MOE_DGA/outputs_gemma/model"
#"""
model, tokenizer = FastModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=2048,
    load_in_4bit=False,
    load_in_8bit = True

)
#"""
# Paso 2: aplica el adaptador LoRA
from peft import PeftModel
model = PeftModel.from_pretrained(model, lora_path)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-03 00:26:18 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-03 00:26:18 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\nThe sequence is the Fibonacci sequence.<end_of_turn>']

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Classify the following domain name as dga or normal: visitform.net",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 10, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The domain name is classified as normal.<end_of_turn>


In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Classify the following domain name as dga or normal: youtube.net",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 10, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 0.7, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The domain name is classified as normal.<end_of_turn>


In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import sys
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import re

families = [
    'matsnu.gz'#,
    #'suppobox.gz',
    #'charbot.gz',
    #'gozi.gz',
    #'manuelita.gz'#,
    #'rovnix.gz',
    #'deception.gz',
    #'nymaim.gz'
]

runs = 30

for family in families:
    print(family)
    dga = pd.read_csv(f'/content/drive/My Drive/Familias_Test/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)
    dfs = []
    for run in range(runs):
        print(f'{run:2}/{runs}', end='\r')
        dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])
        pred = []
        prob = []
        query_time = []
        results = []

        for domain_to_check in dfw.domain.values:
            st = time.time()
            messages = [{
               "role": "user",
               "content": [{
                    "type": "text",
                    "text": f"Classify the following domain name as DGA or normal: {domain_to_check}"
                }]
            }]

            # Generar prompt en formato texto plano
            text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

            # Tokenizar para entrada al modelo
            inputs = tokenizer([text], return_tensors="pt").to("cuda")

            # Generar la salida (sin streamer para capturar resultado)
            output_ids = model.generate(
                **inputs,
                max_new_tokens=128,
                temperature=1.0,
                top_p=0.95,
                top_k=64,
                do_sample=True
            )

            # Decodificar la salida
            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)


            # Ejemplo de salida: "The traffic is classified as normal."
            # Parseamos la respuesta:
            if "classified as dga" in generated_text.lower():
                label_value = 1
            elif "classified as normal" in generated_text.lower():
                label_value = 0
            else:
                # Caso no esperado, asignar 0 o manejar de otra forma
                label_value = 0

            pred.append(label_value)
            query_time.append(time.time() - st)

        dfw['pred'] = pred
        # dfw['prob'] = prob  # Si tienes probabilidades, descomenta esta línea
        dfw['query_time'] = query_time
        dfw.to_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz', index=False)


matsnu.gz


In [ ]:
#"""
families = [
    'matsnu.gz',
    #'suppobox.gz',
    #'charbot.gz',
    #'gozi.gz',
    'manuelita.gz'#,
    #'rovnix.gz',
    #'deception.gz',
    #'nymaim.gz'
]
#"""
def fpr_tpr(y, ypred):
    tn, fp, fn, tp = confusion_matrix(y, ypred).ravel()
    fpr = fp / (fp + tn)  # False Positive Rate
    tpr = tp / (tp + fn)  # True Positive Rate (Recall)
    return fpr, tpr

for family in families:
    acc = []
    pre = []
    rec = []
    f1 = []
    fpr = []
    tpr = []
    qt = []
    qts = []
    for run in range(runs):
        df = pd.read_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz')
        y = (df.label == 'dga').astype(int)
        ypred = df.pred
        acc.append(accuracy_score(y, ypred))
        pre.append(precision_score(y, ypred))
        rec.append(recall_score(y, ypred))
        f1.append(f1_score(y, ypred))
        fpr_value, tpr_value = fpr_tpr(y, ypred)
        fpr.append(fpr_value)
        tpr.append(tpr_value)
        qt.append(df.query_time.mean())
        qts.append(df.query_time.std())
#    print(f'Query time: {np.mean(qt):0.5f}+/-{np.mean(qts)}:0.5f')
    print(f'{family.split(".")[0]:15}: acc:{np.mean(acc):0.2f}±{np.std(acc):.3f} f1:{np.mean(f1):0.2f}±{np.std(f1):.3f} pre:{np.mean(pre):0.2f}±{np.std(pre):.3f} rec:{np.mean(rec):0.2f}±{np.std(rec):.3f}  FPR:{np.mean(fpr):0.2f}±{np.std(fpr):.3f} TPR:{np.mean(tpr):0.2f}±{np.std(tpr):.3f} Query time: {np.mean(qt):0.5f}±{np.mean(qts):0.5f}')


matsnu         : acc:0.92±0.023 f1:0.91±0.026 pre:0.97±0.023 rec:0.86±0.039  FPR:0.02±0.021 TPR:0.86±0.039 Query time: 1.47852±1.03295
manuelita      : acc:0.57±0.025 f1:0.29±0.067 pre:0.87±0.101 rec:0.17±0.046  FPR:0.03±0.024 TPR:0.17±0.046 Query time: 1.35918±1.16398
